In [1]:
import math
import matplotlib.pyplot as plt

In [360]:
class Element_Pricing_Model_Old:
    # input: number of base tokens to buy/sell
    # output: number of fy tokens bought/sold with fee, number of fy tokens bought/sold without fee, fy token fee
    @staticmethod
    def calc_in_given_out(out,in_reserves,out_reserves,g,t):
        k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
        without_fee = pow(k-pow(out_reserves-out,1-t),1/(1-t)) - in_reserves
        #print((without_fee/out,in_reserves,out_reserves))
        if in_reserves > out_reserves:
            # this condition is true when in is the fyt
            fee =  (1-out/without_fee)*(1-g)
            #print((out/without_fee,1/pow(in_reserves/out_reserves,t)))
        else:
            fee =  (1-without_fee/out)*(1-g)
            #print((without_fee/out,pow(out_reserves/in_reserves,t)))
        with_fee = without_fee + fee*out
        return (with_fee,without_fee,fee*out)
    
    # input: number of fy tokens to buy/sell
    # output: number of base tokens bought/sold with fee, number of base tokens bought/sold without fee, base token fee    
    @staticmethod
    def calc_out_given_in(in_,in_reserves,out_reserves,g,t):
        k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
        without_fee = in_reserves - pow(k-pow(out_reserves+in_,1-t),1/(1-t))
        #print((without_fee/in_,in_reserves,out_reserves))
        if in_reserves > out_reserves:
            # this condition is true when in is the fyt
            fee =  (1-in_/without_fee)*(1-g)
            #print((in_/without_fee,1/pow(in_reserves/out_reserves,t)))
        else:
            fee =  (1-without_fee/in_)*(1-g)
            #print((without_fee/in_,pow(out_reserves/in_reserves,t)))
        with_fee = without_fee - fee*in_
        return (with_fee,without_fee,fee*in_)

class Element_Pricing_Model:
    # input: number of base tokens to buy/sell
    # output: number of fy tokens bought/sold with fee, number of fy tokens bought/sold without fee, fy token fee
    @staticmethod
    def calc_in_given_out(out,in_reserves,out_reserves,g,t):
        k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
        without_fee = pow(k-pow(out_reserves-out,1-t),1/(1-t)) - in_reserves
        #print((without_fee/out,in_reserves,out_reserves))
        if in_reserves > out_reserves:
            # this condition is true when in is the fyt
            implied_rate=1-1/pow(in_reserves/out_reserves,t)
            fee =  implied_rate*(1-g)
            #print((out/without_fee,implied_rate))
        else:
            implied_rate=1-1/pow(out_reserves/in_reserves,t)
            fee =  implied_rate*(1-g)
            #print((without_fee/out,implied_rate))
        with_fee = without_fee + fee*without_fee
        return (with_fee,without_fee,fee*without_fee)
    
    # input: number of fy tokens to buy/sell
    # output: number of base tokens bought/sold with fee, number of base tokens bought/sold without fee, base token fee    
    @staticmethod
    def calc_out_given_in(in_,in_reserves,out_reserves,g,t):
        k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
        without_fee = in_reserves - pow(k-pow(out_reserves+in_,1-t),1/(1-t))
        #print((without_fee/in_,in_reserves,out_reserves))
        if in_reserves > out_reserves:
            # this condition is true when in is the fyt
            implied_rate=1-1/pow(in_reserves/out_reserves,t)
            fee =  implied_rate*(1-g)
            #print((in_/without_fee,implied_rate))
        else:
            implied_rate=1-1/pow(out_reserves/in_reserves,t)
            fee =  implied_rate*(1-g)
            #print((without_fee/in_,implied_rate))
        with_fee = without_fee - fee*without_fee
        return (with_fee,without_fee,fee*without_fee)    
    
### Yield Space Method
# x^(1-t) + y^(1-t) = k
# -> y = (k - x^(1-t))^1/(1-t)

# (x-i)^(1-t) + (y+j)^(1-t) = k
# Sell j fyt for i base
# i = x - (k - (y+j)^(1-tg))^1/(1-tg)
# Buy i base for j fyt
# j = (k - (x-i)^(1-t/g))^1/(1-t/g) - y

class Yield_Pricing_Model:
    # input: number of base tokens to buy/sell
    # output: number of fy tokens bought/sold with fee, number of fy tokens bought/sold without fee, fy token fee
    @staticmethod
    def base_to_fyt(amount,x,y,g,t):
        k=pow(x,1-t/g) + pow(y,1-t/g)
        # calculate the number of fyts the specified amount of base tokens will cost with fees
        output_with_fee = pow(k-pow(x-amount,1-t/g),1/(1-t/g)) - y
        k=pow(x,1-t) + pow(y,1-t)
        # calculate the number of fyts the specified amount of base tokens will cost without fees
        output_without_fee = pow(k-pow(x-amount,1-t),1/(1-t)) - y
        # output_with_fee is greater than output_without_fee bc the fee adds additional cost
        fee =  output_with_fee - output_without_fee
        return (output_with_fee,output_without_fee,fee)

    # input: number of fy tokens to buy/sell
    # output: number of base tokens bought/sold with fee, number of base tokens bought/sold without fee, base token fee    
    @staticmethod
    def fyt_to_base(amount,x,y,g,t):
        k=pow(x,1-t*g) + pow(y,1-t*g)
        # calculate the number of base tokens the specified amount of fy tokens will cost with fees
        output_with_fee = x - pow(k-pow(y+amount,1-t*g),1/(1-t*g))
        k=pow(x,1-t) + pow(y,1-t)
        # calculate the number of base tokens the specified amount of fy tokens will cost without fees
        output_without_fee = x - pow(k-pow(y+amount,1-t),1/(1-t))
        # output_with_fee is greater than output_without_fee bc the fee adds additional cost
        fee =  output_with_fee - output_without_fee
        return (output_with_fee,output_without_fee,fee)
    

class Market: 
    def __init__(self,x,y,g,t,pricing_model): 
        self.x=x
        self.y=y
        self.total_supply = x + y
        self.g=g
        self.t=t
        self.pricing_model=pricing_model
        self.x_orders = 0
        self.y_orders = 0
        self.x_volume = 0
        self.y_volume = 0
        self.cum_y_fees=0
        self.cum_x_fees=0
        self.starting_fyt_price=self.fyt_price()
    
    def apy(self):
        return ((self.y+self.total_supply)/self.x - 1) * 100
    
    def fyt_price(self):
        return 1/pow((self.y+self.total_supply)/self.x,self.t)
    
    def tick(self,step_size):
        self.t -= step_size
    
    def swap(self, amount, token_in, token_out):
        output_with_fee = 0
        output_without_fee = 0
        fee = 0
        if token_in == "base" and token_out == "fyt":
            (output_with_fee,output_without_fee,fee) = self.buy_base_with_fyt(amount)
            self.x_orders+=1
            self.x_volume+=amount
        elif token_in == "fyt" and token_out == "base":
            (output_with_fee,output_without_fee,fee) = self.buy_fyt_with_base(amount)
            self.y_orders+=1
            self.y_volume+=amount
        return (output_with_fee,output_without_fee,fee)
        
    # input: amount of base token to buy
    def buy_base_with_fyt(self,amount):
        (output_with_fee,output_without_fee,fee) = self.pricing_model.calc_in_given_out(amount,self.y+self.total_supply,self.x,self.g,self.t)
        self.x -= amount
        self.y += output_with_fee
        self.cum_y_fees += fee
        return (output_with_fee,output_without_fee,fee)
    
    # input: amount of fy token to buy
    def buy_fyt_with_base(self,amount):
        (output_with_fee,output_without_fee,fee) = self.pricing_model.calc_out_given_in(amount,self.x,self.y+self.total_supply,self.g,self.t)
        self.x += output_with_fee
        self.y -= amount
        self.cum_x_fees += fee
        return (output_with_fee,output_without_fee,fee)


In [366]:

display(Element_Pricing_Model.calc_in_given_out(1,5000+55000,50000,.95,.9))
display(Element_Pricing_Model.calc_in_given_out(1,50000,5000+55000,.95,.9))


display(Element_Pricing_Model.calc_out_given_in(1,50000,5000+55000,.95,.9))
display(Element_Pricing_Model.calc_out_given_in(1,5000+55000,50000,.95,.9))

1/pow(505/500,.9)

(1.18725681364719, 1.178340671933256, 0.008916141713933923)

(0.8551006871368296, 0.8486789940216113, 0.006421693115218361)

(0.842231801299508, 0.8486532999959309, 0.006421498696422825)

(1.1693828121545211, 1.178298635793908, 0.008915823639386842)

0.9910846814801719

In [361]:
display(Element_Pricing_Model_Old.calc_in_given_out(10,5000+55000,50000,.95,.9))
display(Element_Pricing_Model_Old.calc_in_given_out(10,50000,5000+55000,.95,.9))


display(Element_Pricing_Model_Old.calc_out_given_in(10,50000,5000+55000,.95,.9))
display(Element_Pricing_Model_Old.calc_out_given_in(10,5000+55000,50000,.95,.9))

(11.861041343244993, 11.78529872661602, 0.07574261662897382)

(8.563549053399038, 8.487946371998987, 0.07560268140005073)

(8.409645798856582, 8.485376951291983, 0.07573115243540092)

(11.705503858812472, 11.781095095495402, 0.07559123668293029)

In [362]:
import numpy as np

runs={}
np.random.seed(1)


x_start = 50000
y_start = 5000
g=.95
t=.9
step_size=.0005
m = Market(x_start,y_start,g,t,Element_Pricing_Model)
print("Starting APY: " + str(m.apy()))
print("Starting FYT Price: " + str(m.fyt_price()))
print("Starting X Reserves: " + str(m.x))
print("Starting Y Reserves: " + str(m.y))
x_orders=0
x_volume=0
y_orders=0
y_volume=0
buy_base_with_fyt = True
output = []
while m.t-step_size >= 0:
    # determine order size
    amount = np.random.uniform(0,10)
    # buy fyt or base
    if np.random.uniform(0,1) < 0.5:
        token_in = "base"
        token_out = "fyt"
    else:
        token_in = "fyt"
        token_out = "base"
    swap_input = {
        "time": m.t,
        "x_reserves": m.x,
        "y_reserves": m.y,
        "fee_percent": g,
        "token_in": token_in,
        "amount_in": amount,
        "token_out": token_out
    }
    (output_with_fee,output_without_fee,fee) = m.swap(amount,token_in,token_out)
    swap_output = {
        "x_reserves": m.x,
        "y_reserves": m.y,
        "amount_out": output_with_fee,
        "fee": fee
    }
    
    output.append({
        "input": swap_input,
        "output": swap_output
    });
    m.tick(step_size)

print("Ending X Reserves: " + str(m.x))
print("Ending Y Reserves: " + str(m.y))
print("Num x orders: " + str(m.x_orders))
print("Cum x volume: " + str(m.x_volume))
print("Num y orders: " + str(m.y_orders))
print("Cum y volume: " + str(m.y_volume))
print("Cum fees x: " + str(m.cum_x_fees))
print("Cum fees y: " + str(m.cum_y_fees))
print("Ending APY: " + str(m.apy()))
print("Ending FYT Price: " + str(m.fyt_price()))
print("Ending Time: " + str(m.t))

runs[1]=output

Starting APY: 19.999999999999996
Starting FYT Price: 0.8486661467891868
Starting X Reserves: 50000
Starting Y Reserves: 5000
Ending X Reserves: 49979.26706668693
Ending Y Reserves: 5028.437355131177
Num x orders: 884
Cum x volume: 4285.002128842804
Num y orders: 916
Cum y volume: 4646.757151883774
Cum fees x: 16.408543258655673
Cum fees y: 18.568874656928546
Ending APY: 20.106677985164765
Ending FYT Price: 0.999999999999992
Ending Time: 4.3635234314720606e-14


In [363]:
import numpy as np

runs={}
np.random.seed(1)


x_start = 50000
y_start = 5000
g=.95
t=.9
step_size=.0005
m = Market(x_start,y_start,g,t,Element_Pricing_Model_Old)
print("Starting APY: " + str(m.apy()))
print("Starting FYT Price: " + str(m.fyt_price()))
print("Starting X Reserves: " + str(m.x))
print("Starting Y Reserves: " + str(m.y))
x_orders=0
x_volume=0
y_orders=0
y_volume=0
buy_base_with_fyt = True
output = []
while m.t-step_size >= 0:
    # determine order size
    amount = np.random.uniform(0,10)
    # buy fyt or base
    if np.random.uniform(0,1) < 0.5:
        token_in = "base"
        token_out = "fyt"
    else:
        token_in = "fyt"
        token_out = "base"
    swap_input = {
        "time": m.t,
        "x_reserves": m.x,
        "y_reserves": m.y,
        "fee_percent": g,
        "token_in": token_in,
        "amount_in": amount,
        "token_out": token_out
    }
    (output_with_fee,output_without_fee,fee) = m.swap(amount,token_in,token_out)
    swap_output = {
        "x_reserves": m.x,
        "y_reserves": m.y,
        "amount_out": output_with_fee,
        "fee": fee
    }
    
    output.append({
        "input": swap_input,
        "output": swap_output
    });
    m.tick(step_size)

print("Ending X Reserves: " + str(m.x))
print("Ending Y Reserves: " + str(m.y))
print("Num x orders: " + str(m.x_orders))
print("Cum x volume: " + str(m.x_volume))
print("Num y orders: " + str(m.y_orders))
print("Cum y volume: " + str(m.y_volume))
print("Cum fees x: " + str(m.cum_x_fees))
print("Cum fees y: " + str(m.cum_y_fees))
print("Ending APY: " + str(m.apy()))
print("Ending FYT Price: " + str(m.fyt_price()))
print("Ending Time: " + str(m.t))



Starting APY: 19.999999999999996
Starting FYT Price: 0.8486661467891868
Starting X Reserves: 50000
Starting Y Reserves: 5000
Ending X Reserves: 49977.36330400266
Ending Y Reserves: 5026.551453460355
Num x orders: 884
Cum x volume: 4285.002128842804
Num y orders: 916
Cum y volume: 4646.757151883774
Cum fees x: 18.30436757325558
Cum fees y: 16.673797555993257
Ending APY: 20.107479636991687
Ending FYT Price: 0.999999999999992
Ending Time: 4.3635234314720606e-14


In [353]:
#%pip install genson
import json
from genson import SchemaBuilder

with open('test_vectors.json', 'w') as fp:
    json.dump(runs, fp, indent=1)
    
builder = SchemaBuilder()
builder.add_object(runs)
runs_schema=builder.to_schema()

with open('test_vectors_schema.json', 'w') as fp:
    json.dump(runs_schema, fp, indent=1)

In [66]:
import numpy as np

runs={}
np.random.seed(1)


x_start = 50000
y_start = 5000
g=.95
t=.9
step_size=.0005
m = Market(x_start,y_start,g,t,Element_Pricing_Model)
print("Starting APY: " + str(m.apy()))
print("Starting FYT Price: " + str(m.fyt_price()))
print("Starting X Reserves: " + str(m.x))
print("Starting Y Reserves: " + str(m.y))
x_orders=0
x_volume=0
y_orders=0
y_volume=0
buy_base_with_fyt = True
output = []
while m.t-step_size >= 0:
    # determine order size
    amount = np.random.uniform(0,100)
    # buy fyt or base
    if np.random.uniform(0,1) < 0.5:
        token_in = "base"
        token_out = "fyt"
    else:
        token_in = "fyt"
        token_out = "base"
    swap_input = {
        "time": m.t,
        "x_reserves": m.x,
        "y_reserves": m.y,
        "fee_percent": g,
        "token_in": token_in,
        "amount_in": amount,
        "token_out": token_out
    }
    (output_with_fee,output_without_fee,fee) = m.swap(amount,token_in,token_out)
    swap_output = {
        "x_reserves": m.x,
        "y_reserves": m.y,
        "amount_out": output_with_fee,
        "fee": fee
    }
    
    output.append({
        "input": swap_input,
        "output": swap_output
    });
    m.tick(step_size)

print("Ending X Reserves: " + str(m.x))
print("Ending Y Reserves: " + str(m.y))
print("Num x orders: " + str(m.x_orders))
print("Cum x volume: " + str(m.x_volume))
print("Num y orders: " + str(m.y_orders))
print("Cum y volume: " + str(m.y_volume))
print("Cum fees x: " + str(m.cum_x_fees))
print("Cum fees y: " + str(m.cum_y_fees))
print("Ending APY: " + str(m.apy()))
print("Ending FYT Price: " + str(m.fyt_price()))
print("Ending Time: " + str(m.t))

runs[2]=output

Starting APY: 19.999999999999996
Starting FYT Price: 0.8486661467891868
Starting X Reserves: 50000
Starting Y Reserves: 5000
Ending X Reserves: 50043.19912328182
Ending Y Reserves: 5337.817006434054
Num x orders: 884
Cum x volume: 42850.02128842794
Num y orders: 916
Cum y volume: 46467.57151883775
Cum fees x: 188.1237424804223
Cum fees y: 169.86679213173412
Ending APY: 20.571462383512618
Ending FYT Price: 0.9999999999999918
Ending Time: 4.3635234314720606e-14


In [67]:
import numpy as np

runs={}
np.random.seed(1)


x_start = 50000
y_start = 5000
g=.95
t=.9
step_size=.0005
m = Market(x_start,y_start,g,t,Yield_Pricing_Model)
print("Starting APY: " + str(m.apy()))
print("Starting FYT Price: " + str(m.fyt_price()))
print("Starting X Reserves: " + str(m.x))
print("Starting Y Reserves: " + str(m.y))
x_orders=0
x_volume=0
y_orders=0
y_volume=0
buy_base_with_fyt = True
output = []
while m.t-step_size >= 0:
    # determine order size
    amount = np.random.uniform(0,10)
    # buy fyt or base
    if np.random.uniform(0,1) < 0.5:
        token_in = "base"
        token_out = "fyt"
    else:
        token_in = "fyt"
        token_out = "base"
    swap_input = {
        "time": m.t,
        "x_reserves": m.x,
        "y_reserves": m.y,
        "fee_percent": g,
        "token_in": token_in,
        "amount_in": amount,
        "token_out": token_out
    }
    (output_with_fee,output_without_fee,fee) = m.swap(amount,token_in,token_out)
    swap_output = {
        "x_reserves": m.x,
        "y_reserves": m.y,
        "amount_out": output_with_fee,
        "fee": fee
    }
    
    output.append({
        "input": swap_input,
        "output": swap_output
    });
    m.tick(step_size)

print("Ending X Reserves: " + str(m.x))
print("Ending Y Reserves: " + str(m.y))
print("Num x orders: " + str(m.x_orders))
print("Cum x volume: " + str(m.x_volume))
print("Num y orders: " + str(m.y_orders))
print("Cum y volume: " + str(m.y_volume))
print("Cum fees x: " + str(m.cum_x_fees))
print("Cum fees y: " + str(m.cum_y_fees))
print("Ending APY: " + str(m.apy()))
print("Ending FYT Price: " + str(m.fyt_price()))
print("Ending Time: " + str(m.t))

runs[3]=output

Starting APY: 19.999999999999996
Starting FYT Price: 0.8486661467891868
Starting X Reserves: 50000
Starting Y Reserves: 5000
Ending X Reserves: 50013.625814334184
Ending Y Reserves: 5029.853125288396
Num x orders: 884
Cum x volume: 4285.002128842804
Num y orders: 916
Cum y volume: 4646.757151883774
Cum fees x: 17.341368952344055
Cum fees y: 20.668325990198355
Ending APY: 20.026996939069175
Ending FYT Price: 0.999999999999992
Ending Time: 4.3635234314720606e-14


In [69]:
import numpy as np

runs={}
np.random.seed(1)


x_start = 50000
y_start = 5000
g=.95
t=.9
step_size=.0005
m = Market(x_start,y_start,g,t,Yield_Pricing_Model)
print("Starting APY: " + str(m.apy()))
print("Starting FYT Price: " + str(m.fyt_price()))
print("Starting X Reserves: " + str(m.x))
print("Starting Y Reserves: " + str(m.y))
x_orders=0
x_volume=0
y_orders=0
y_volume=0
buy_base_with_fyt = True
output = []
while m.t-step_size >= 0:
    # determine order size
    amount = np.random.uniform(0,100)
    # buy fyt or base
    if np.random.uniform(0,1) < 0.5:
        token_in = "base"
        token_out = "fyt"
    else:
        token_in = "fyt"
        token_out = "base"
    swap_input = {
        "time": m.t,
        "x_reserves": m.x,
        "y_reserves": m.y,
        "fee_percent": g,
        "token_in": token_in,
        "amount_in": amount,
        "token_out": token_out
    }
    (output_with_fee,output_without_fee,fee) = m.swap(amount,token_in,token_out)
    swap_output = {
        "x_reserves": m.x,
        "y_reserves": m.y,
        "amount_out": output_with_fee,
        "fee": fee
    }
    
    output.append({
        "input": swap_input,
        "output": swap_output
    });
    m.tick(step_size)

print("Ending X Reserves: " + str(m.x))
print("Ending Y Reserves: " + str(m.y))
print("Num x orders: " + str(m.x_orders))
print("Cum x volume: " + str(m.x_volume))
print("Num y orders: " + str(m.y_orders))
print("Cum y volume: " + str(m.y_volume))
print("Cum fees x: " + str(m.cum_x_fees))
print("Cum fees y: " + str(m.cum_y_fees))
print("Ending APY: " + str(m.apy()))
print("Ending FYT Price: " + str(m.fyt_price()))
print("Ending Time: " + str(m.t))

runs[4]=output

Starting APY: 19.999999999999996
Starting FYT Price: 0.8486661467891868
Starting X Reserves: 50000
Starting Y Reserves: 5000
Ending X Reserves: 50022.648509931736
Ending Y Reserves: 5392.349834961335
Num x orders: 884
Cum x volume: 42850.02128842794
Num y orders: 916
Cum y volume: 46467.57151883775
Cum fees x: 178.93060107397469
Cum fees y: 211.72496327491535
Ending APY: 20.730012572146705
Ending FYT Price: 0.9999999999999918
Ending Time: 4.3635234314720606e-14
